# MARS3888 - Finding and Using Online Datasets 

## Notebook 2


+ Part A: Build multi-file NetCDF dataset
    + Clip and Aggregate dataset
    + Plot map from multi-file NetCDF dataset
+ Part B: Extract data at specificed location from multi-file NetCDF dataset

#### Load the required Python libraries

First of all, load the necessary libraries:

+ numpy
+ matplotlib
+ cartopy
+ xarray
+ cmocean


In [ ]:
import os
import numpy as np
import xarray as xr

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

import cmocean

from matplotlib import pyplot as plt
#%config InlineBackend.figure_format = 'retina'
plt.ion()  # To trigger the interactive inline mode

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# Part A: Build multi-file NetCDF dataset

We will use the `open_mfdataset` function from `xArray` to open multiple netCDF files into a single xarray Dataset. 

+ Many datesets are separted into daily, monthly or even annual NetCDFs so being able to combine them makes it a lot easier to analyse or plot the data.

We will query load the GBR4km dataset from the [AIMS server](http://thredds.ereefs.aims.gov.au/thredds/catalog.html), so let's first define the base URL:

In [ ]:
# For the bio dataset
base_url = "http://thredds.ereefs.aims.gov.au/thredds/dodsC/s3://aims-ereefs-public-prod/derived/ncaggregate/ereefs/GBR4_H2p0_B3p1_Cq3b_Dhnd/daily-monthly/EREEFS_AIMS-CSIRO_GBR4_H2p0_B3p1_Cq3b_Dhnd_bgc_daily-monthly-"

# For the hydro dataset
base_url2 = "http://thredds.ereefs.aims.gov.au/thredds/dodsC/s3://aims-ereefs-public-prod/derived/ncaggregate/ereefs/gbr4_v2/daily-monthly/EREEFS_AIMS-CSIRO_gbr4_v2_hydro_daily-monthly-"

For the sake of the demonstration, we will only use 1 month:

In [38]:
month_st = 1   # Starting month 
month_ed = 1   # Ending month 
year = 2018    # Year

# Based on the server the file naming convention 
biofiles = [f"{base_url}{year}-{month:02}.nc" for month in range(month_st, month_ed+1)]
hydrofiles = [f"{base_url2}{year}-{month:02}.nc" for month in range(month_st, month_ed+1)]
biofiles

['http://thredds.ereefs.aims.gov.au/thredds/dodsC/s3://aims-ereefs-public-prod/derived/ncaggregate/ereefs/GBR4_H2p0_B3p1_Cq3b_Dhnd/daily-monthly/EREEFS_AIMS-CSIRO_GBR4_H2p0_B3p1_Cq3b_Dhnd_bgc_daily-monthly-2018-01.nc']

## Loading dataset into xArray

Using `xArray`, we open these files into a `Dataset`:

In [ ]:
ds_bio = xr.open_mfdataset(biofiles)
ds_hydro = xr.open_mfdataset(hydrofiles)
ds_hydro

# Clip and Aggregate Dataset 

When working with large NetCDF files it is useful to know how to clip the data to an area or depth of interest.

This part of the notebook is an extension of what we have done in Notebook 1.

### Clip by Coordinates (X and Y)

To reduce the `Xarray Dataset` size we will clip the spatial extent based on longitudinal and latitudinal values.

+ As we already saw this is easely done using the `sel` function with the slice method (see Notebook 1).

In [ ]:
min_lon = 150     # lower left longitude
min_lat = -27     # lower left latitude
max_lon = 155     # upper right longitude
max_lat = -20     # upper right latitude

# Defining the boundaries
lon_bnds = [min_lon, max_lon]
lat_bnds = [min_lat, max_lat]

# Performing the reduction
ds_hydro_clip = ds_hydro.sel(latitude=slice(*lat_bnds), longitude=slice(*lon_bnds))
ds_hydro_clip

### Clip by Depth (Z)

For this example we will use the surface values (i.e. the last z-coordinate at -0.5 m), this is done by using `isel` function on the `Xarray Dataset`:

In [ ]:
zcvar = -1  # last z-coordinate at 0.5 m

new_ds = ds_hydro_clip.isel(k=zcvar)
new_ds

# Aggregate Data using `Groupby` 

Using `Groupby` ([more info here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)) with time coordinates, we take the maximum of each data variables over a 2 weeks timeframe (‘2W’). We also reduce the Dataset to the following variables:

+ ‘temp’,
+ ‘mean_cur’,
+ ‘u’, and
+ ‘v’

In [ ]:
week_ds = new_ds.resample(time='2W').max(dim='time').drop('zc')[['temp','mean_cur','u','v']]
week_ds

You now have the basic skills to open and reduce NetCDF files.

# Plot map from multi-file NetCDF dataset

Using the `ds_hydro` Dataset lets plot a map of `ds_hydro.temp` for the first day in the dataset:

In [ ]:
# specify and check the date to plot
day= 1 # change this value to the number index in `time`

We want to plot a Marker for the location of One Tree Island:

In [ ]:
reef_lat = -23.508
reef_lon = 152.091

In [ ]:
# Figure size
size = (9, 10)

# Color from cmocean
color = cmocean.cm.balance

# Defining the figure
fig = plt.figure(figsize=size, facecolor='w', edgecolor='k')

# Axes with Cartopy projection
ax = plt.axes(projection=ccrs.PlateCarree())
# and extent
ax.set_extent([142.4, 157, -7, -28.6], ccrs.PlateCarree())

# Plotting using Matplotlib 
# We plot the Temp at the surface at the final recorded time interval
cf = ds_hydro.temp.isel(time=day,k=-1).plot( 
    transform=ccrs.PlateCarree(), cmap=color,
    vmin = 22, vmax = 32,
    add_colorbar=False
)

# Color bar
cbar = fig.colorbar(cf, ax=ax, fraction=0.027, pad=0.045, 
                    orientation="horizontal")
cbar.set_label(ds_hydro.temp.long_name+' '+ds_hydro.temp.units, rotation=0, 
               labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

# Title
plt.title('Sea Surface Temperature (C) on ' +str(ds_hydro.coords['time'].values[day])[:10],
          fontsize=11
         )

# Plot lat/lon grid 
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.1, color='k', alpha=1, 
                  linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 8}
gl.ylabel_style = {'size': 8} 

# Add map features with Cartopy 
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', 
                                            edgecolor='face', 
                                            facecolor='lightgray'))

ax.coastlines(linewidth=1)

# Site One Tree Island
ax.scatter(reef_lon, reef_lat, c='deeppink', s=50, edgecolors='k', linewidth=1, transform=ccrs.PlateCarree())

plt.tight_layout()
plt.show()
fig.clear()
plt.close(fig)
plt.clf()

If you want to plot another day in the dataset you must change the `day` value

# Part B: Find closest xArray data for a reef location

We will use One Tree Island position for the example

In [ ]:
reef_lat = -23.508
reef_lon = 152.091

Using the `sel` function and `nearest` method we extract the values of the variables close to our site:

In [ ]:
# Alkalinity
alk = ds_bio.alk
alk = alk.sel(longitude=reef_lon, latitude=reef_lat, method='nearest')

# Mean current
mean_cur = ds_hydro.mean_cur
mean_cur = mean_cur.sel(longitude=reef_lon, latitude=reef_lat, method='nearest')

Let's check where the closest point is located in the eReefs dataset to One Tree Island:

In [ ]:
lat = alk.latitude.values.item(0)
lon = alk.longitude.values.item(0)

print('Nearest position: ',lon,lat)

We then load the `Xarray Dataset` in memory:

This may take a while to load:

In [ ]:
load_alk = alk.load()
load_curr = mean_cur.load()

# Plot timeseries of eReef data from closest Xarray data point to One Tree

We will use the `Xarray` `plot` function.

Plot alkalinity at One Tree Island:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

n = load_alk.zc.shape[0]  # Number of z-coordinate

# Colormap from cmocean discretized based on zc number 
colors = cmocean.cm.matter(np.linspace(0,1,n))

# We use matplotlib-backed plotting interface to get
# the alkalinity evolution over time for each z-coordinate
for zc in range(n):
    load_alk.isel(k=zc).plot(ax=ax, x='time', lw=3, 
                             color=colors[zc], 
                             label='zc='+str(load_alk.zc.item(zc))+' m')

# Add legend
ax.legend(
    bbox_to_anchor=(1.05, 1), loc='upper left', 
    fontsize=10, frameon=False
)

# Define x/y axis
ax.set_xlim(min(load_alk.time.values),max(load_alk.time.values))
ax.set_ylabel(load_alk.long_name+' in '+load_alk.units, style="italic", fontsize=12)
ax.set_xlabel("Date", fontsize=12)

ax.grid(True, linewidth=0.5, color="k", alpha=0.1, linestyle="-")
ax.tick_params(labelcolor="k", labelsize="medium", width=3)
plt.title('eReefs total alkalinity at One Tree Reef for variable depths', fontsize=13)

plt.tight_layout()

Plot currents at One Tree Island:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

n = load_curr.zc.shape[0]  # Number of z-coordinate

# Colormap from cmocean discretized based on zc number 
colors = cmocean.cm.matter(np.linspace(0,1,n))

# We use matplotlib-backed plotting interface to get
# the alkalinity evolution over time for each z-coordinate
for zc in range(n):
    load_curr.isel(k=zc).plot(ax=ax, x='time', lw=3, 
                             color=colors[zc], 
                             label='zc='+str(load_curr.zc.item(zc))+' m')

# Add legend
ax.legend(
    bbox_to_anchor=(1.05, 1), loc='upper left', 
    fontsize=10, frameon=False
)

# Define x/y axis
ax.set_xlim(min(load_curr.time.values),max(load_curr.time.values))
ax.set_ylabel(load_curr.long_name+' in '+load_curr.units, style="italic", fontsize=12)
ax.set_xlabel("Date", fontsize=12)

ax.grid(True, linewidth=0.5, color="k", alpha=0.1, linestyle="-")
ax.tick_params(labelcolor="k", labelsize="medium", width=3)
plt.title('eReefs currents at One Tree Reef for variable depths', fontsize=13)

plt.tight_layout()

Now you know the basics of reading and plotting multi-file NetCDF data.

### End of Notebook 2